In [10]:
using DifferentialEquations
using Plots
using Interact
using Statistics
using LinearAlgebra

#using Pkg
#Pkg.add("WebIO")
#using WebIO



In [11]:
include("aux.jl")

df (generic function with 1 method)

In [27]:
function theorem_holds(my_param)
    err_msg = []
    holds = my_param.υ>0 && my_param.ρ>0
    
    for i=1:2
        holds = holds && (my_param.xstar[i][end]<1) && (my_param.xstar[i][1]<1)
    end

       
    holds = holds && all( all(diff(my_params.β[i]).>0)  for i=1:2) #betas elements are increasing
    if !all( all(diff(my_params.β[i]).>0)  for i=1:2)
        push!(err_msg, "betas elements are NOT increasing!")
    end
    
    holds = holds && all( all(diff(my_params.c[i]).<0)  for i=1:2)     #costs elements are increasing
    if !all( all(diff(my_params.c[i]).<0)  for i=1:2) 
        push!(err_msg, "costs vector elements are NOT increasing!") 
    end
    
    
    holds = holds && minimum(diag(diagm([my_params.β[i][1] for i=1:2])*my_params.M))>my_params.δ+my_params.θ+my_params.γ #assumption 1
    if !(minimum(diag(diagm([my_params.β[i][1] for i=1:2])*my_params.M))>my_params.δ+my_params.θ+my_params.γ)
        push!(err_msg, "assumption 1 does NOT hold !")
    end
        
    #assumption 2
    show_once = true
    for i=1:2
        h(β,c,k) = (c[k]-c[k+1])/(β[k+1]-β[k])
        β = my_params.β[i]
        c = my_params.c[i]
        for j=1:(my_params.SIZE_n[i]-2)
            @assert h(β,c,j)>=h(β,c,j+1) "error on i=$(i), c: j=$(j)"
             holds = holds && h(β,c,j)<h(β,c,j+1)
            if show_once && !( h(β,c,j)<h(β,c,j+1))
                push!(err_msg, "assumption 2 does NOT hold !")
                show_once = false
                
            end
        end
    end

    
    (holds,err_msg)
end

theorem_holds (generic function with 1 method)

In [28]:
function perturb_xstar_r!(my_params,cstar,ρ)
    @assert sum( my_params.xstar[j]'*my_params.c[j] for j=1:2)<=cstar
    @assert ρ>0
    
    
    ϵ = 0.002
    δ = 2*ϵ
    
    global xstar = deepcopy(my_params.xstar) 
    meets_budget = false
    while(!meets_budget)
        δ /= 2.0
        for i=1:2
            if xstar[i][end] == 1
                xstar[i][end]   -= δ
                xstar[i][end-1] += δ
            else 
                for j=1:my_params.SIZE_n[i]-1
                    if  xstar[i][j] == 1
                        xstar[i][j]   -= ϵ
                        xstar[i][j+1] += ϵ
                    else  xstar[i][j]>0 && xstar[i][j+1]>0
                        local s = min(ϵ,0.9*xstar[i][j],0.9*(1-xstar[i][j+1]))
                        xstar[i][j]   -= s
                        xstar[i][j+1] += s
                    end
                end
            end
        end
           
        meets_budget = (sum( xstar[j]'*my_params.c[j] for j=1:2)/cstar)<1.0
    end
    
    
    my_params.xstar = deepcopy(xstar)
    for i=1:2
        my_params.r[i] = -(xstar[i].==0).*ρ
    end
    
end

perturb_xstar_r! (generic function with 1 method)

In [29]:
mutable struct SIS_E2PG
        SIZE_q
        SIZE_n # number of strategies per population - a tuple of integers >0    
        θ 
        γ
        δ
        M # contact rate matrix, in the paper it is capital Θ
        β
        r
        c
        υ
        xstar
        ρ

        function SIS_E2PG() 

        SIZE_q = 2 # number of populations - must be 2
        SIZE_n = (2,2) # number of strategies per population - a tuple of integers >0    
        θ = Float64(0.0002)
        γ = Float64(0.14)
        δ = Float64(0.0005)
        M = Float64.([1.0 0.3; 0.1 1.0]) # contact rate matrix, in the paper it is capital Θ
        β = [   Float64.([0.15;0.19]), 
            Float64.([0.15;0.19]) ]
        r = [   Float64.([0.15;0.15]),
                Float64.([0.15]) ]
        c = [   Float64.([0.35;0.0]),
                Float64.([0.40;0.0]) ]
        υ = 4.0
        xstar = []
        ρ = 0.01
        new(SIZE_q, SIZE_n, θ, γ, δ, M, β, r, c, υ, xstar, ρ)
        end
end


my_params = SIS_E2PG() 

SIS_E2PG(2, (2, 2), 0.0002, 0.14, 0.0005, [1.0 0.3; 0.1 1.0], [[0.15, 0.19], [0.15, 0.19]], [[0.15, 0.15], [0.15]], [[0.35, 0.0], [0.4, 0.0]], 4.0, Any[], 0.01)

In [30]:
I0   = Vector{Float64}([0.001; 0.001])
x0_1 = Vector{Float64}([1.0;0.0])
x0_2 = Vector{Float64}([1.0;0.0])
q0   = Vector{Float64}([0.0;0.0])

2-element Vector{Float64}:
 0.0
 0.0

In [31]:
my_params.M = Float64.([1.0 0.3; 0.1 1.0])

2×2 Matrix{Float64}:
 1.0  0.3
 0.1  1.0

In [32]:
all(diff(my_params.c[2]).<0)

true

In [39]:
R = 0:0.01:1.0
@manipulate for θ in (R.*0.009 .+0.0001), 
    γ in (R.*0.14 .+0.01), 
    δ in (R.*0.009 .+0.0001), 
    cstar in (R.*.5 .+0.0001), 
    i1 in (R.*0.999 .+0.001), 
    i2 in (R.*0.999 .+0.001), 
    x011=R, 
    x021=R, 
    do_remark2_approx=Dict("yes"=>true, "no"=>false)
    
    
    my_params.θ = θ
    my_params.γ = γ
    my_params.δ = δ
    
    I0[1] = i1
    I0[2] = i2
    
    x0_1[1]= x011
    x0_2[1]= x021
    x0_1[2] = 1-x0_1[1]
    x0_2[2] = 1-x0_2[1]
        
    my_params.xstar=find_xstar(my_params,cstar)
    my_params.r=find_rΔ(my_params.xstar)
    
    ## if approx_it then find an xstar for which the theorem works
    if do_remark2_approx
        perturb_xstar_r!(my_params,cstar,1.0)
    end
    
    u0 = [I0,[x0_1,x0_2],q0];
    prob = ODEProblem(df,formatted2vec(u0),[0.0,1500],my_params)
    sol = solve(prob,AutoTsit5(Rosenbrock23()), save_everystep=true, saveat=1.0)
     
    a = plot(sol, idxs=[1,2], label=[L"I_1" L"I_2"], title="Infected", ylims=[0,0.40])
    b = plot(sol, idxs=[3,5], label=[L"x_1^1" L"x_1^2"], title="Social State, υ=$(my_params.υ)")
    
    
    t1 = plot(grid=false, axis=false, ticks=false)
        annotate!(0.52, 0.0, "Final", :left) 
        annotate!(0.52, 0.17, L"$I^* = $"*"$(round.(find_equilibrium_I(my_params,[[],my_params.xstar,[]] );digits=4))", :left) 
        annotate!(0.52, 0.34, L"$x_1^* = $"* "$(round.(my_params.xstar[1];digits=4))", :left)
        annotate!(0.52, 0.51, L"$x_2^* = $"* "$(round.(my_params.xstar[2];digits=4))", :left) 
        annotate!(0.52, 0.68, L"$c'x^* = $"* "$(round(sum( my_params.c[i]'*my_params.xstar[i] for i=1:2);digits=4))", :left)
        annotate!(0.52, 0.85, L"$\lambda_{\max}$ = "* "$(round(eigmax(B(my_params, [[],my_params.xstar,[]]));digits=4))", :left)
    #
        annotate!(0.05, 0.0, "Initial", :left)
        annotate!(0.05, 0.17, L"$I(0)$ ="*" $(round.(I0;digits=4))", :left) 
        annotate!(0.05, 0.34, L"$X_1(0)$  = "*"$(round.(x0_1;digits=2))", :left)
        annotate!(0.05, 0.51, L"$X_2(0)$  = "*"$(round.(x0_2;digits=2))", :left)
    
        annotate!(0.15, 1.0, text("Theorem holds:", :black, 20), :left, color=:red)
        
        the_theorem_holds,err_msg = theorem_holds(my_params)
        if the_theorem_holds
             annotate!(0.35, 1.02, text("■", :green, 30), :left)
        else
             annotate!(0.35, 1.02, text("■", :red, 30), :left)
             k = 0.0 
             for m in err_msg
                 annotate!(0.15, 0.87-k, text(m, :red, 10), :left)
                 k = k + 0.09
        end
        end
       
    
    plot(a,b,t1,
        size=(800,550),
        #layout = @layout [a ; b c]
        layout = @layout [[a b]; c]   
    )
    ylims!(0.0,1.1)


end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["θ"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 101, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable{Any}(51), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/certorio/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/certorio/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/certorio/.julia/packages/InteractBase/Qhvxg/src/../assets/all.js"), Asset("css", nothing, "/home/certorio/.julia/packages/InteractBase/Qhvxg/src/../assets/style.css"), Asset("css", nothing, "/home/certorio/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f4490df42f0, Task (runnable) @0x00007f4490df42f0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0001\",\"0.00019\",\"0.00028\",\"0.00037\",\"0.00046\",\"0.00055\",\"0.00064\",\"0.00073\",\"0.00082\",\"0.00091\",\"0.001\",\"0.00109\",\"0.00118\",\"0.00127\",\"0.00136\",\"0.00145\",\"0.00154\",\"0.00163\",\"0.00172\",\"0.00181\",\"0.0019\",\"0.00199\",\"0.00208\",\"0.00217\",\"0.00226\",\"0.00235\",\"0.00244\",\"0.00253\",\